In [1]:
import pandas as pd
import sqlite3

# Load the data from CSV files
users = pd.read_csv("data/USER_TAKEHOME.csv")
transactions = pd.read_csv("data/TRANSACTION_TAKEHOME.csv")
products = pd.read_csv("data/PRODUCTS_TAKEHOME.csv")

# Convert date fields
users["CREATED_DATE"] = pd.to_datetime(users["CREATED_DATE"], errors="coerce")
users["BIRTH_DATE"] = pd.to_datetime(users["BIRTH_DATE"], errors="coerce")
transactions["PURCHASE_DATE"] = pd.to_datetime(transactions["PURCHASE_DATE"], errors="coerce")
transactions["SCAN_DATE"] = pd.to_datetime(transactions["SCAN_DATE"], errors="coerce")

# Convert numeric fields
transactions["FINAL_QUANTITY"] = pd.to_numeric(transactions["FINAL_QUANTITY"], errors="coerce")
transactions["FINAL_SALE"] = pd.to_numeric(transactions["FINAL_SALE"], errors="coerce")

# Fill missing values
users.fillna({"STATE": "Unknown", "LANGUAGE": "Unknown", "GENDER": "Unknown"}, inplace=True)
products.fillna({"CATEGORY_1": "Unknown", "CATEGORY_2": "Unknown", "CATEGORY_3": "Unknown", 
                "CATEGORY_4": "Unknown", "MANUFACTURER": "Unknown", "BRAND": "Unknown"}, inplace=True)

# Save cleaned data
users.to_csv("data/CLEANED_USERS.csv", index=False)
transactions.to_csv("data/CLEANED_TRANSACTIONS.csv", index=False)
products.to_csv("data/CLEANED_PRODUCTS.csv", index=False)

print("\nUser Analsis : \n",users.head())
print("\nTransaction Analsis : \n",transactions.head())
print("\nProducts Analsis : \n",products.head())



User Analsis : 
                          ID              CREATED_DATE  \
0  5ef3b4f17053ab141787697d 2020-06-24 20:17:54+00:00   
1  5ff220d383fcfc12622b96bc 2021-01-03 19:53:55+00:00   
2  6477950aa55bb77a0e27ee10 2023-05-31 18:42:18+00:00   
3  658a306e99b40f103b63ccf8 2023-12-26 01:46:22+00:00   
4  653cf5d6a225ea102b7ecdc2 2023-10-28 11:51:50+00:00   

                 BIRTH_DATE STATE LANGUAGE   GENDER  
0 2000-08-11 00:00:00+00:00    CA   es-419   female  
1 2001-09-24 04:00:00+00:00    PA       en   female  
2 1994-10-28 00:00:00+00:00    FL   es-419   female  
3                       NaT    NC       en  Unknown  
4 1972-03-19 00:00:00+00:00    PA       en   female  

Transaction Analsis : 
                              RECEIPT_ID PURCHASE_DATE  \
0  0000d256-4041-4a3e-adc4-5623fb6e0c99    2024-08-21   
1  0001455d-7a92-4a7b-a1d2-c747af1c8fd3    2024-07-20   
2  00017e0a-7851-42fb-bfab-0baa96e23586    2024-08-18   
3  000239aa-3478-453d-801e-66a82e39c8af    2024-06-18   
4  00

In [2]:
# Connect to SQLite database (or create one)
conn = sqlite3.connect("data/mystore.db")
cursor = conn.cursor()

# Load DataFrames into SQLite
users.to_sql("users", conn, if_exists="replace", index=False)
transactions.to_sql("transactions", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)

# Define SQL queries
queries = {
    "Top 5 Brands by Receipts Scanned (Users 21+)": """
        WITH user_age AS (
            SELECT 
                ID, 
                (strftime('%Y', 'now') - strftime('%Y', BIRTH_DATE)) AS age
            FROM users
        )
        SELECT 
            p.BRAND, 
            COUNT(t.RECEIPT_ID) AS receipt_count
        FROM transactions t
        JOIN user_age u ON t.USER_ID = u.ID
        JOIN products p ON t.BARCODE = p.BARCODE
        WHERE u.age >= 21
        GROUP BY p.BRAND
        ORDER BY receipt_count DESC
        LIMIT 5;
    """,
    
    "Top Brand in 'Dips & Salsa' Category": """
        SELECT p.BRAND, COUNT(*) AS total_sales
        FROM products p
        JOIN transactions t ON p.BARCODE = t.BARCODE
        WHERE p.CATEGORY_2 = 'Dips & Salsa'
        GROUP BY p.BRAND
        ORDER BY total_sales DESC
        LIMIT 1;
    """,
    
    "Identifying Fetch’s Power Users": """
        SELECT 
            t.USER_ID, 
            COUNT(DISTINCT t.RECEIPT_ID) AS total_receipts, 
            SUM(t.FINAL_SALE) AS total_spent
        FROM transactions t
        GROUP BY t.USER_ID
        ORDER BY total_spent DESC
        LIMIT 10;
    """,
    
    "Generation-wise Sales Percentage in 'Health & Wellness'": """
        SELECT 
            CASE 
                WHEN (strftime('%Y', 'now') - strftime('%Y', u.BIRTH_DATE)) BETWEEN 18 AND 25 THEN 'Gen Z'
                WHEN (strftime('%Y', 'now') - strftime('%Y', u.BIRTH_DATE)) BETWEEN 26 AND 41 THEN 'Millennials'
                WHEN (strftime('%Y', 'now') - strftime('%Y', u.BIRTH_DATE)) BETWEEN 42 AND 57 THEN 'Gen X'
                ELSE 'Boomers & Older'
            END AS Generation,
            COUNT(t.BARCODE) * 100.0 / SUM(COUNT(t.BARCODE)) OVER() AS percentage_sales
        FROM users u
        JOIN transactions t ON u.ID = t.USER_ID
        JOIN products p ON t.BARCODE = p.BARCODE
        WHERE p.CATEGORY_1 = 'Health & Wellness'
        GROUP BY Generation;
    """
}

# Execute each query and print results
for title, query in queries.items():
    print(f"\n--- {title} ---\n")
    result = pd.read_sql_query(query, conn)
    print(result.to_string(index=False))

# Close database connection
conn.close()


--- Top 5 Brands by Receipts Scanned (Users 21+) ---

          BRAND  receipt_count
        Unknown              6
    NERDS CANDY              6
           DOVE              6
        TRIDENT              4
SOUR PATCH KIDS              4

--- Top Brand in 'Dips & Salsa' Category ---

   BRAND  total_sales
TOSTITOS           72

--- Identifying Fetch’s Power Users ---

                 USER_ID  total_receipts  total_spent
630789e1101ae272a4852287               1       925.64
63af23db9f3fc9c7546fdbec               1       476.34
650874eafe41d365c2ee11d2               1       267.29
645add3bffe0d7e043ef1b63               4       227.93
637257e75fdbb03aa198a310               4       194.14
63ba250039c79dcbdd4fbc6c               1       187.04
6535138fdda97e6b1342ea7f               3       146.65
610b3d5002b8c34bbf3a589d               2       144.96
5d61b8e71ddc4058bd98f776               1       143.94
5e022fe1daccde120544eb42               3       140.74

--- Generation-wise Sales Perce